# Clustering of transcripts upstream of TSS

Work on clustering transcripts using their conservation score 1 kb upstream of TSS:
- slice
- intersect w/conservation scores
- 10 bp bins
- avg score w/in bins
- table
- standardize
- cluster! 

In [1]:
import gffutils
from gffutils import pybedtools_integration
import pybedtools
from pybedtools.featurefuncs import gff2bed
import pandas as pd
import seaborn as sb
import statsmodels
import statsmodels.api as sm
%matplotlib inline

/data/bergeric/miniconda3/envs/s2rnai/lib/python3.5/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


Import database and define transcripts, genes, tsses, etc. 

In [37]:
db = gffutils.FeatureDB('/data/LCDB/lcdb-references/dmel/r6-11/gtf/dmel_r6-11.gtf.db')

/data/bergeric/miniconda3/envs/s2rnai/lib/python3.5/site-packages/gffutils/interface.py:161: UserWarning: It appears that this database has not had the ANALYZE sqlite3 command run on it. Doing so can dramatically speed up queries, and is done by default for databases created with gffutils >0.8.7.1 (this database was created with version 0.8.7.1) Consider calling the analyze() method of this object.
  "method of this object." % self.version)


In [38]:
transcripts = pybedtools_integration.to_bedtool(db.features_of_type('transcript')).saveas()

In [39]:
genes = pybedtools_integration.to_bedtool(db.features_of_type('gene')).sort().merge().saveas()

In [40]:
tss = pybedtools_integration.tsses(db, merge_overlapping=False).saveas('../../output/tsses.gff')

In [41]:
slop = tss.slop(l=1000, r=0, s=True, genome='dm6').saveas('../../output/another_slop.bed')

Get intersections TSS w/exons, TSS w/introns, TSS w/intergenic:

In [11]:
TSS_exons = tss.intersect('../../output/dm6_exons.bed').saveas().to_dataframe()

In [21]:
introns = pd.read_table('../../output/dmel-introns-r6.11.gff', header=None)

In [27]:
introns[0] = [ 'chr'+x for x in introns[0] ] 

In [35]:
tss_introns = tss.intersect(pybedtools.BedTool.from_dataframe(introns), wa=True).saveas().to_dataframe()

In [16]:
TSS_intergenic = tss.intersect('../../output/intergenic.bed').saveas().to_dataframe()

Intersect upstream of TSS w/conservation scores:

(If I try to do here will fail, probably because I don't have enough memory? so doing it on local machine)

In [ ]:
#wig2bed < dm6.27way.phastCons.wigFix > dm6_phastcons.bed
phastcons = pybedtools.BedTool('../../output/dm6_phastcons.bed').saveas()

## 2L 

In [42]:
#import intersect file: (really big, splitting by chrom)
intersect_2L = pd.read_table('../../output/2L_intersect', header=None)

In [43]:
intersect_2L.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,chr2L,gffutils_derived,transcript_TSS,6529,7529,.,+,.,"gene_id ""FBgn0031208""; transcript_id ""FBtr0300...",chr2L,6528,6529,id-3194,0.021,0
1,chr2L,gffutils_derived,transcript_TSS,6529,7529,.,+,.,"gene_id ""FBgn0031208""; transcript_id ""FBtr0300...",chr2L,6529,6530,id-3195,0.020,0
2,chr2L,gffutils_derived,transcript_TSS,6529,7529,.,+,.,"gene_id ""FBgn0031208""; transcript_id ""FBtr0300...",chr2L,6530,6531,id-3196,0.023,0
3,chr2L,gffutils_derived,transcript_TSS,6529,7529,.,+,.,"gene_id ""FBgn0031208""; transcript_id ""FBtr0300...",chr2L,6531,6532,id-3197,0.024,0
4,chr2L,gffutils_derived,transcript_TSS,6529,7529,.,+,.,"gene_id ""FBgn0031208""; transcript_id ""FBtr0300...",chr2L,6532,6533,id-3198,0.025,0


NEED 10bp bins (i.e. TSS-1…-10 = bin1, TSS-11…-20 = bin2 …) :: 

In [87]:
done=[]
table = []
grp = intersect_2L.groupby([3,4])
x = 0 
for name, group in grp:
    if name not in done:
        done.append(name)
        grp_num = x
        for i in range(name[0], name[1]):
            print(i)
            break
        bin_start = name[0]
        bin_end = name[0] + 10
        table.append([grp_num, bin_start, bin_end])
        x+=1
        
        

6529
18570
20952
21376
25155
53817
59268
64999
65318
65404
65482
66043
70757
71387
72485
73129
73836
75348
87387
93739
101380
105903
106732
106761
106937
107089
107133
108594
113726
116182
116488
120631
120742
121754
121837
129245
130791
131077
134402
135144
139588
140992
142323
142572
143091
153300
154089
154335
155029
155149
155305
158032
158664
181820
202779
206179
206406
206797
207297
218651
221962
222519
227037
231487
232253
247909
250823
252373
254750
261203
267505
268088
271744
272836
272937
273715
276582
277212
278234
282807
283195
283269
285383
285726
286531
288932
289144
291419
291795
294122
294681
296880
300706
304935
309200
309348
319259
321101
321248
329761
332335
354663
356329
357874
377112
395120
402054
402198
403285
412885
414067
414777
418952
419652
420771
421450
421873
430227
438539
451302
452108
452752
453024
453754
454531
471555
472013
472200
475220
479688
489835
490879
502236
522466
522736
542580
544129
544653
549502
552454
552923
558612
559079
560310
563163
563347

5336170
5338073
5342234
5345200
5381074
5386551
5403280
5403633
5430842
5431822
5434236
5437749
5439277
5460639
5485905
5516737
5519135
5519260
5519896
5523139
5523298
5523898
5524905
5525957
5526943
5527123
5532223
5536758
5539196
5541480
5541542
5541779
5541998
5542100
5542314
5546086
5546203
5546290
5550835
5550838
5552897
5553850
5554072
5558342
5564619
5565755
5581116
5588414
5595833
5614230
5617946
5620280
5623762
5632660
5658560
5661849
5662232
5677582
5689529
5691421
5705374
5707391
5708950
5710903
5714678
5715701
5716680
5720051
5720540
5720899
5723656
5728811
5740954
5741244
5745157
5766142
5768752
5772516
5773598
5783031
5800173
5800253
5801220
5801975
5804395
5808702
5809352
5810574
5818443
5823004
5823794
5879498
5882741
5883888
5883932
5884774
5885002
5887149
5887989
5892862
5893896
5900380
5904689
5906169
5906250
5910289
5923653
5924937
5925961
5939033
5940995
5941246
5942643
5942965
5943284
5943304
5948397
5948450
5956004
5956466
5956819
5958631
5960340
5964322
5964951


10645415
10649857
10650613
10656627
10657281
10661081
10676198
10676365
10676817
10680030
10680480
10680902
10680966
10684409
10686989
10687066
10689271
10694058
10694103
10698043
10701544
10702621
10706190
10709101
10709453
10723203
10724515
10727978
10729255
10731653
10732423
10736071
10736345
10737043
10743935
10756325
10760116
10767086
10767939
10768006
10769868
10769931
10769952
10786173
10789222
10801109
10802801
10818107
10819705
10826958
10837511
10842597
10844402
10845664
10847044
10849065
10849141
10850340
10850848
10851511
10856644
10857536
10858094
10864697
10869167
10869867
10871014
10885454
10888927
10895344
10901785
10903959
10908893
10910541
10910648
10935662
10936353
10942949
10943622
10964153
10966623
10966811
10968166
10969443
10970203
10970275
10971280
10971598
10975293
10975772
10976363
10976399
10986806
10987420
10988401
11001072
11001108
11001476
11003820
11004153
11004189
11010312
11011086
11012135
11014620
11015995
11033795
11057464
11066998
11070027
11071466
1

16679054
16679570
16681503
16682402
16684008
16684390
16694524
16697761
16700825
16703197
16714891
16714961
16716195
16718932
16721011
16727826
16728777
16729681
16731039
16731550
16732133
16737850
16738388
16742405
16743100
16753336
16760138
16760554
16765566
16765734
16765926
16766777
16767611
16790568
16790721
16793211
16795595
16798025
16798768
16803884
16804077
16806316
16815660
16816108
16821093
16825207
16826078
16826359
16833998
16834502
16836972
16837258
16837591
16839137
16847197
16848716
16852368
16858562
16858714
16860497
16861489
16868346
16868462
16869490
16878517
16879290
16880212
16887490
16887507
16887966
16888908
16900473
16902218
16910418
16945374
16950070
16964475
16972091
16981483
16982301
16983978
16997327
16997683
17013134
17014700
17018049
17043165
17048068
17048281
17084650
17086641
17104145
17125961
17130024
17170412
17172481
17174763
17175618
17177772
17179338
17180938
17246161
17246617
17260734
17279235
17285240
17292202
17311820
17319308
17320712
17353877
1

21492423
21492562
21493294
21493875
21494691
21495586
21497468
21497607
21498339
21498920
21499736
21500633
21502521
21502660
21503392
21503973
21504789
21505684
21507566
21507705
21508437
21509018
21509834
21510728
21512610
21512749
21513481
21514062
21514878
21515773
21517655
21517794
21518526
21519109
21520083
21520616
21522498
21522637
21523369
21523952
21524926
21525459
21527341
21527480
21528212
21528795
21529769
21530302
21532184
21532323
21533055
21533636
21534610
21535346
21537228
21537367
21538099
21540153
21541127
21541864
21572389
21577676
21589423
21590593
21618774
21623627
21623679
21624080
21627462
21629932
21631587
21656625
21656659
21657735
21664849
21670197
21671513
21676227
21677230
21677530
21680700
21682116
21682164
21683106
21753723
21757010
21757466
21784065
21795447
21799112
21818490
21827593
21854094
21866267
21866291
21868020
21870652
21891095
21893227
21898289
21901993
21905289
21915300
21915358
21928064
21949874
21950521
21956820
21957463
21959980
21992933
2

In [ ]:
#hopefully this will be done running by tomorrow and I can get the table this way: (Unless its huge...)
phastwindows = pd.read_table('../../output/phastwindows_2L')